In [1]:
import time
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
import re

In [2]:
class Data:
    def __init__(self,name,profit,day,hours,amount,method):
        self.name= name
        self.profit= profit
        self.day= pd.to_datetime(day)
        self.amount = amount
        self.method = method
        self.hours = hours
        
    def create_data_object():
        name,profit,day,hours,amount,method = input('Name of data? '), input('How much did you make? '),input('Date of profit? '),input('How many hours did this take? '),input('What was the amount that created this profit? '),input('What type of method was this? (Bossing,Slayer,Skiling, etc)')
        data = Data(name,profit,day,hours,amount,method)
        return data 
    
    def create_dict(self):
        return dict(Name = [self.name],
             Profit = [self.profit],
             Day= [self.day],
             Amount = [self.amount],
             Method= [self.method],
             Hours = [self.hours]
            )       

In [53]:
def add_item():
    data = Data.create_data_object()
    df = pd.DataFrame(data.create_dict())
    total_df.append(df)
    df = pd.concat(total_df,ignore_index=True)
    return df

def sort_item(dataframe):
    print('How would you like to search through your data?\n1.Name\n2.Profit\n3.Day\n4.Amount\n5.Method\n6.Hours')
    number = int(input('Number: '))
    sort = columns[number - 1]
    return dataframe.sort_values(by = sort, ascending = False)

def edit_item(sorted_df):
    item = input('Which item do you want to edit: ')
    column = input('Which value do you want to edit: ')
    new_value = input('What is the new value: ')
    sorted_df.loc[item,column] = new_value
    return sorted_df.reset_index(drop=True)
    
def delete_item(sorted_df):
    item = input('What item do you want to delete?')
    for i in range(len(columns)):
        index = sorted_df[sorted_df[columns[i]] == item].index
        if len(index) > 1:
            print('Mulitple %s logs found.' % item)
            print(sorted_df[sorted_df[columns[i]] == item])
            delete = int(input('Which entry number would you like to delete?'))
            sorted_df.drop(delete,inplace=True)
            sorted_df.reset_index(drop=True,inplace =True)
            return sorted_df
        elif len(index) == 1:
            sorted_df.drop(sorted_df[sorted_df[columns[i]] == item].index,inplace=True)
            sorted_df.reset_index(drop=True,inplace =True)
            return sorted_df
        else:
            continue

In [4]:
total_df = []
columns = ['Name','Profit','Day','Amount','Method','Hours']

In [59]:
while True:
    print("What would you like to do?\n1.Add Data\n2.Edit data\n3.Delete data\n4.End ")
    answer= input("")
    if answer == '1':
        df = add_item()
    elif answer == '2':
        df = sort_item(df)
        print(df)
        df = edit_item(df)
    elif answer == '3':
        df = sort_item(df)
        print(df)
        df = delete_item(df)
        print(df)
    elif answer == '4':
        break

What would you like to do?
1.Add Data
2.Edit data
3.Delete data
4.End 
3
How would you like to search through your data?
1.Name
2.Profit
3.Day
4.Amount
5.Method
6.Hours
Number: 3
       Name       Profit        Day Amount   Method    Hours
8         a    000000000 2019-08-21   2019      ETC        8
4    random           34 2019-08-16     58      ETC  3333333
2   vorkath         4934 2019-08-15  34444  Bossing        4
7         a          888 2019-08-15      8      ETC        9
6         a     99999999 2019-08-12      8      ETC      999
1   vorkath           99 2019-08-10     22  Bossing        9
0   vorkath  44444444444 2019-08-07     88  Bossing    99999
3    random          889 2019-08-06     10      ETC        9
5  asdfasdf       888888 2019-08-05      3      ETC        a
9         a           43 2019-08-04    200      ETC       89
What item do you want to delete?vorkath
Mulitple vorkath logs found.
      Name       Profit        Day Amount   Method  Hours
2  vorkath         4934

False

In [60]:
df

,Name,Profit,Day,Amount,Method,Hours
0,a,000000000,2019-08-21,2019,ETC,8
1,random,34,2019-08-16,58,ETC,3333333
2,a,888,2019-08-15,8,ETC,9
3,a,99999999,2019-08-12,8,ETC,999
4,vorkath,99,2019-08-10,22,Bossing,9
5,vorkath,44444444444,2019-08-07,88,Bossing,99999
6,random,889,2019-08-06,10,ETC,9
7,asdfasdf,888888,2019-08-05,3,ETC,a
8,a,43,2019-08-04,200,ETC,89
